In [65]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
from datetime import datetime, timedelta
import math
import asyncio
import coinbase
from coinbase.candle import candle

In [82]:
r = await candle("ADA-USD", 60, start = datetime(2021, 11, 29))

In [84]:
r[0].json()

[[1638199560, 1.5696, 1.5717, 1.5717, 1.57, 58071.5],
 [1638199500, 1.5688, 1.5719, 1.5688, 1.5719, 43475.76],
 [1638199440, 1.5673, 1.5694, 1.5682, 1.569, 102124.67],
 [1638199380, 1.5676, 1.5693, 1.5688, 1.5682, 7616.18],
 [1638199320, 1.5681, 1.5695, 1.5691, 1.569, 83401.17],
 [1638199260, 1.5689, 1.5707, 1.57, 1.5691, 28255.37],
 [1638199200, 1.5698, 1.5723, 1.5717, 1.57, 34917.14],
 [1638199140, 1.57, 1.5717, 1.5706, 1.5716, 32636.59],
 [1638199080, 1.5695, 1.5707, 1.5697, 1.5704, 17067.13],
 [1638199020, 1.568, 1.57, 1.568, 1.5697, 24824.4],
 [1638198960, 1.5678, 1.5688, 1.5682, 1.568, 35765.23],
 [1638198900, 1.566, 1.5688, 1.5665, 1.5682, 34581.05],
 [1638198840, 1.5661, 1.5696, 1.5678, 1.5661, 69179.95],
 [1638198780, 1.5669, 1.5687, 1.5685, 1.5678, 18066],
 [1638198720, 1.5681, 1.5694, 1.5691, 1.5685, 32716.65],
 [1638198660, 1.5683, 1.5703, 1.5695, 1.5689, 57076.83],
 [1638198600, 1.568, 1.5696, 1.5684, 1.5696, 37583.05],
 [1638198540, 1.5674, 1.5687, 1.5677, 1.5687, 69252.9

In [15]:
api.client.aclose()

NameError: name 'api' is not defined

In [43]:
api = coinbase.Market()

In [44]:
r = await api.coins()
r

<Response [200 OK]>

In [47]:
r.json()

[{'id': 'POLY-USD',
  'base_currency': 'POLY',
  'quote_currency': 'USD',
  'base_min_size': '1',
  'base_max_size': '1400000',
  'quote_increment': '0.0001',
  'base_increment': '1',
  'display_name': 'POLY/USD',
  'min_market_funds': '1',
  'max_market_funds': '100000',
  'margin_enabled': False,
  'fx_stablecoin': False,
  'max_slippage_percentage': '0.03000000',
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'trading_disabled': False,
  'status': 'online',
  'status_message': '',
  'auction_mode': False},
 {'id': 'CRO-USDT',
  'base_currency': 'CRO',
  'quote_currency': 'USDT',
  'base_min_size': '1',
  'base_max_size': '1300000',
  'quote_increment': '0.0001',
  'base_increment': '0.1',
  'display_name': 'CRO/USDT',
  'min_market_funds': '1',
  'max_market_funds': '100000',
  'margin_enabled': False,
  'fx_stablecoin': False,
  'max_slippage_percentage': '0.03000000',
  'post_only': False,
  'limit_only': True,
  'cancel_only': False,
  'trading_disabled': 